# 04. 지도 해석 및 정책 우선순위

## 목적
- 지역별 위험도 집계
- 정책 우선순위 지역 도출

In [1]:
import pandas as pd

result_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"
df = pd.read_csv(f"{result_dir}/클러스터링_결과_완전판.csv")

data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
full_df = pd.read_csv(data_path)

df = df.merge(full_df[['가맹점구분번호', '지역명', '상권_코드_명']].drop_duplicates(), on='가맹점구분번호', how='left')

print(f"데이터: {len(df):,}개")

데이터: 4,325개


## 지역별 위험도 분석

In [2]:
region_stats = df.groupby('지역명').agg(
    총가맹점수=('가맹점구분번호', 'count'),
    클러스터0_수=('cluster', lambda x: (x == 0).sum()),
    클러스터2_수=('cluster', lambda x: (x == 2).sum()),
    D3m_수=('상태', lambda x: (x == 'D-3m').sum())
).reset_index()

region_stats['고위험_수'] = region_stats['클러스터0_수'] + region_stats['클러스터2_수']
region_stats['고위험_비율'] = region_stats['고위험_수'] / region_stats['총가맹점수']

region_stats = region_stats.sort_values('고위험_비율', ascending=False)

print("[지역별 위험도 TOP 10]")
print(region_stats[['지역명', '총가맹점수', '고위험_수', '고위험_비율', 'D3m_수']].head(10))

[지역별 위험도 TOP 10]
   지역명  총가맹점수  고위험_수    고위험_비율  D3m_수
1  약수역    150    150  1.000000      0
0   뚝섬   1649   1635  0.991510      8
2  왕십리   1519   1495  0.984200     14
4  청량리    769    753  0.979194      6
3  장안동    238    231  0.970588      4


### 정책적 시사점
- 고위험 비율이 높은 지역: 집중 지원 필요
- D-3m 가맹점: 긴급 개입 대상